# Group 5 - Module 2: Recommendation Systems

***
## Group Members:
* ### Nils Dunlop, 20010127-2359, Applied Data Science, e-mail: gusdunlni@student.gu.se (16 hours)
* ### Francisco Alejandro Erazo Piza, 19930613-9214, Applied Data Science, e-mail: guserafr@student.gu.se (16 hours)

#### **We hereby declare that we have both actively participated in solving every exercise. All solutions are entirely our own work, without having taken part of other solutions." (This is independent and additional to any declaration that you may encounter in the electronic submission system.)**

# Assignment 3
***

## Problem 1: Reading and Reflection From The Hidden Technical Debt in Machine Learning Systems paper.
***

## Problem 2: Implementation
***

## Problem 3: Discussion
***

## References:
***

## Self Check
***
- Have you answered all questions to the best of your ability?
Yes, we have.
- Is all the required information on the front page, is the file name correct etc.?
Indeed, all the required information on the front page has been included.
- Anything else you can easily check? (details, terminology, arguments, clearly stated answers etc.?)
We have checked, and everything looks good.